In [66]:
! pip install category_encoders

     ---------------------------------------- 81.9/81.9 kB 4.8 MB/s eta 0:00:00


In [1]:
import numpy as np 
import seaborn as sns
import pandas as pd

In [2]:
df=sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [3]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [4]:
df=df[[  'sex', 'age', 'who', 'parch', 'fare', 'class',  'embark_town','alive', 'alone']]
df

,sex,age,who,parch,fare,class,embark_town,alive,alone
0,male,22.0,man,0,7.2500,Third,Southampton,no,False
1,female,38.0,woman,0,71.2833,First,Cherbourg,yes,False
2,female,26.0,woman,0,7.9250,Third,Southampton,yes,True
3,female,35.0,woman,0,53.1000,First,Southampton,yes,False
4,male,35.0,man,0,8.0500,Third,Southampton,no,True
...,...,...,...,...,...,...,...,...,...
886,male,27.0,man,0,13.0000,Second,Southampton,no,True
887,female,19.0,woman,0,30.0000,First,Southampton,yes,True
888,female,NaN,woman,2,23.4500,Third,Southampton,no,False
889,male,26.0,man,0,30.0000,First,Cherbourg,yes,True


In [5]:
df=df.dropna()

In [6]:
x,y=df.drop(columns=['alive']),df['alive']
x

,sex,age,who,parch,fare,class,embark_town,alone
0,male,22.0,man,0,7.2500,Third,Southampton,False
1,female,38.0,woman,0,71.2833,First,Cherbourg,False
2,female,26.0,woman,0,7.9250,Third,Southampton,True
3,female,35.0,woman,0,53.1000,First,Southampton,False
4,male,35.0,man,0,8.0500,Third,Southampton,True
...,...,...,...,...,...,...,...,...
885,female,39.0,woman,5,29.1250,Third,Queenstown,False
886,male,27.0,man,0,13.0000,Second,Southampton,True
887,female,19.0,woman,0,30.0000,First,Southampton,True
889,male,26.0,man,0,30.0000,First,Cherbourg,True


In [7]:
catcolumns= [ele for ele in x.columns if x[ele].dtype]

In [8]:
x.select_dtypes(include=['object'])

,sex,who,embark_town
0,male,man,Southampton
1,female,woman,Cherbourg
2,female,woman,Southampton
3,female,woman,Southampton
4,male,man,Southampton
...,...,...,...
885,female,woman,Queenstown
886,male,man,Southampton
887,female,woman,Southampton
889,male,man,Cherbourg


In [9]:
xcat=x.select_dtypes(include=['object','category','bool'])
xcont=x.select_dtypes(exclude=['object','category','bool'])

In [26]:
xcatOrdinal=xcat['class']
xcatNominal=xcat.drop(columns=['class'])

In [27]:
## split data to train and test data
import sklearn
from sklearn.model_selection import train_test_split

In [62]:
xcontTrain,xcontTest,xcatOrdinalTrain,xcatOrdinalTest,xcatNominalTrain,xcatNominalTest,yTrain,yTest=train_test_split(xcont,xcatOrdinal,xcatNominal,y,test_size=0.32,random_state=23)

In [47]:
## cont
## robust scaling--> step1--> (fit_transform on xcontTrain  & transfrom on xcontTest)
## box-cox --> step 2 --> (fit_transform on xcontTrain & transform on xcont Test)

In [48]:
from sklearn.preprocessing import RobustScaler

In [49]:
rscaler= RobustScaler()

In [50]:
## fit_ transform 
xcontTrain=pd.DataFrame(rscaler.fit_transform(xcontTrain),columns=xcontTrain.columns,index=xcontTrain.index)
xcontTrain

,age,parch,fare
257,0.115942,0.0,2.715179
780,-0.869565,0.0,-0.296057
106,-0.405797,0.0,-0.280072
51,-0.405797,0.0,-0.274374
450,0.463768,2.0,0.483461
...,...,...,...
294,-0.231884,0.0,-0.270735
40,0.695652,0.0,-0.210746
620,-0.057971,0.0,-0.021603
54,2.144928,1.0,1.783715


In [51]:
xcontTest=pd.DataFrame(rscaler.transform(xcontTest),columns=xcontTest.columns,index=xcontTest.index)
xcontTest

,age,parch,fare
38,-0.579710,0.0,0.113090
315,-0.115942,0.0,-0.272315
336,0.057971,0.0,1.959244
605,0.463768,0.0,0.020023
193,-1.449275,1.0,0.416984
...,...,...,...
606,0.115942,0.0,-0.270735
816,-0.289855,0.0,-0.269626
514,-0.231884,0.0,-0.285930
876,-0.463768,0.0,-0.196661


In [52]:
from sklearn.preprocessing import PowerTransformer

In [53]:
pTransformer=PowerTransformer()

In [54]:
xcontTrain=pd.DataFrame(pTransformer.fit_transform(xcontTrain),columns=xcontTrain.columns,index=xcontTrain.index)
xcontTrain

,age,parch,fare
257,0.090997,-0.614757,1.490741
780,-1.198370,-0.614757,-1.133356
106,-0.566929,-0.614757,-1.074268
51,-0.566929,-0.614757,-1.053543
450,0.495599,1.717865,0.503245
...,...,...,...
294,-0.340678,-0.614757,-1.040399
40,0.753829,-0.614757,-0.833940
620,-0.121127,-0.614757,-0.301241
54,2.233520,1.533523,1.263841


In [55]:
xcontTest=pd.DataFrame(pTransformer.transform(xcontTest),columns=xcontTest.columns,index=xcontTest.index)
xcontTest

,age,parch,fare
38,-0.799215,-0.614757,-0.018557
315,-0.193518,-0.614757,-1.046098
336,0.021120,-0.614757,1.317023
605,0.495599,-0.614757,-0.206270
193,-2.035449,1.533523,0.428857
...,...,...,...
606,0.090997,-0.614757,-1.040399
816,-0.415387,-0.614757,-1.036407
514,-0.340678,-0.614757,-1.095758
876,-0.643716,-0.614757,-0.788201


In [56]:
xcatTrain.columns

Index(['age', 'parch', 'fare'], dtype='object')

In [57]:
xcatOrdinalTrain

,age,parch,fare
257,30.0,0,86.5000
780,13.0,0,7.2292
106,21.0,0,7.6500
51,21.0,0,7.8000
450,36.0,2,27.7500
...,...,...,...
294,24.0,0,7.8958
40,40.0,0,9.4750
620,27.0,0,14.4542
54,65.0,1,61.9792


In [58]:
from sklearn.preprocessing import OrdinalEncoder
ordenc=OrdinalEncoder()

In [63]:
xcatOrdinalTrain=xcatOrdinalTrain.map({'first':2,
                'second':1,
                'third':0})

In [65]:
xcatOrdinalTest=xcatOrdinalTest.map({'first':2,
                'second':1,
                'third':0})

In [67]:
import category_encoders 
from category_encoders import BinaryEncoder

In [71]:
binenc=BinaryEncoder(cols=xcatNominalTrain.columns)

In [106]:
xcatTrainNominalBnaryenc=binenc.fit_transform(xcatNominalTrain)

In [107]:
xcatTrainNominalBnaryenc

,sex_0,sex_1,who_0,who_1,embark_town_0,embark_town_1,alone_0,alone_1
257,0,1,0,1,0,1,0,1
780,0,1,1,0,1,0,0,1
106,0,1,0,1,0,1,0,1
51,1,0,1,1,0,1,0,1
450,1,0,1,1,0,1,1,0
...,...,...,...,...,...,...,...,...
294,1,0,1,1,0,1,0,1
40,0,1,0,1,0,1,1,0
620,1,0,1,1,1,0,1,0
54,1,0,1,1,1,0,1,0


In [104]:
xcatNominalTestBinaryenc=binenc.transform(xcatNominalTest)

In [108]:
xcatNominalTestBinaryenc

,sex_0,sex_1,who_0,who_1,embark_town_0,embark_town_1,alone_0,alone_1
38,0,1,0,1,0,1,1,0
315,0,1,0,1,0,1,0,1
336,1,0,1,1,0,1,1,0
605,1,0,1,1,0,1,1,0
193,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...
606,1,0,1,1,0,1,0,1
816,0,1,0,1,0,1,0,1
514,1,0,1,1,0,1,0,1
876,1,0,1,1,0,1,0,1


In [79]:
## one hot encoding -- leave one out 
from sklearn.preprocessing import OneHotEncoder
oneenc=OneHotEncoder(sparse=False ,drop='first')

In [80]:
oneenc.fit_transform(xcatNominalTrain)

array([[0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 1., 1.],
       ...,
       [1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 1., 0., 1.]])

In [81]:
oneenc.get_feature_names_out()

array(['sex_male', 'who_man', 'who_woman', 'embark_town_Queenstown',
       'embark_town_Southampton', 'alone_True'], dtype=object)

In [85]:
xcatNominalTrainOneHot=pd.DataFrame(oneenc.fit_transform(xcatNominalTrain),columns=oneenc.get_feature_names_out(),
                                    index=xcatNominalTrain.index)
xcatNominalTrainOneHot

,sex_male,who_man,who_woman,embark_town_Queenstown,embark_town_Southampton,alone_True
257,0.0,0.0,1.0,0.0,1.0,1.0
780,0.0,0.0,0.0,0.0,0.0,1.0
106,0.0,0.0,1.0,0.0,1.0,1.0
51,1.0,1.0,0.0,0.0,1.0,1.0
450,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
294,1.0,1.0,0.0,0.0,1.0,1.0
40,0.0,0.0,1.0,0.0,1.0,0.0
620,1.0,1.0,0.0,0.0,0.0,0.0
54,1.0,1.0,0.0,0.0,0.0,0.0


In [87]:
xcatNominalTestOneHot=pd.DataFrame(oneenc.transform(xcatNominalTest),columns=oneenc.get_feature_names_out(),
                                    index=xcatNominalTest.index)
xcatNominalTestOneHot

,sex_male,who_man,who_woman,embark_town_Queenstown,embark_town_Southampton,alone_True
38,0.0,0.0,1.0,0.0,1.0,0.0
315,0.0,0.0,1.0,0.0,1.0,1.0
336,1.0,1.0,0.0,0.0,1.0,0.0
605,1.0,1.0,0.0,0.0,1.0,0.0
193,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
606,1.0,1.0,0.0,0.0,1.0,1.0
816,0.0,0.0,1.0,0.0,1.0,1.0
514,1.0,1.0,0.0,0.0,1.0,1.0
876,1.0,1.0,0.0,0.0,1.0,1.0


In [94]:
df1=pd.concat([xcontTrain,xcatOrdinalTrain,xcatNominalTrainOneHot],axis=1)

In [95]:
df2=pd.concat([xcontTest,xcatOrdinalTest,xcatNominalTestOneHot],axis=1)

In [96]:
df1

,age,parch,fare,class,sex_male,who_man,who_woman,embark_town_Queenstown,embark_town_Southampton,alone_True
257,30.0,0,86.5000,NaN,0.0,0.0,1.0,0.0,1.0,1.0
780,13.0,0,7.2292,NaN,0.0,0.0,0.0,0.0,0.0,1.0
106,21.0,0,7.6500,NaN,0.0,0.0,1.0,0.0,1.0,1.0
51,21.0,0,7.8000,NaN,1.0,1.0,0.0,0.0,1.0,1.0
450,36.0,2,27.7500,NaN,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
294,24.0,0,7.8958,NaN,1.0,1.0,0.0,0.0,1.0,1.0
40,40.0,0,9.4750,NaN,0.0,0.0,1.0,0.0,1.0,0.0
620,27.0,0,14.4542,NaN,1.0,1.0,0.0,0.0,0.0,0.0
54,65.0,1,61.9792,NaN,1.0,1.0,0.0,0.0,0.0,0.0


In [97]:
df2

,age,parch,fare,class,sex_male,who_man,who_woman,embark_town_Queenstown,embark_town_Southampton,alone_True
38,18.0,0.0,18.0000,NaN,0.0,0.0,1.0,0.0,1.0,0.0
315,26.0,0.0,7.8542,NaN,0.0,0.0,1.0,0.0,1.0,1.0
336,29.0,0.0,66.6000,NaN,1.0,1.0,0.0,0.0,1.0,0.0
605,36.0,0.0,15.5500,NaN,1.0,1.0,0.0,0.0,1.0,0.0
193,3.0,1.0,26.0000,NaN,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df3=pd.concat([xcontTrain,xcatTrainNominalBnaryenc,xcatOrdinalTrain],axis=1)


In [112]:
df4=pd.concat([xcontTest,xcatNominalTestBinaryenc,xcatOrdinalTest],axis=1)

In [113]:
df3


,age,parch,fare,sex_0,sex_1,who_0,who_1,embark_town_0,embark_town_1,alone_0,alone_1,class
257,30.0,0,86.5000,0,1,0,1,0,1,0,1,NaN
780,13.0,0,7.2292,0,1,1,0,1,0,0,1,NaN
106,21.0,0,7.6500,0,1,0,1,0,1,0,1,NaN
51,21.0,0,7.8000,1,0,1,1,0,1,0,1,NaN
450,36.0,2,27.7500,1,0,1,1,0,1,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
294,24.0,0,7.8958,1,0,1,1,0,1,0,1,NaN
40,40.0,0,9.4750,0,1,0,1,0,1,1,0,NaN
620,27.0,0,14.4542,1,0,1,1,1,0,1,0,NaN
54,65.0,1,61.9792,1,0,1,1,1,0,1,0,NaN


In [114]:
df4

,age,parch,fare,sex_0,sex_1,who_0,who_1,embark_town_0,embark_town_1,alone_0,alone_1,class
38,18.0,0.0,18.0000,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN
315,26.0,0.0,7.8542,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,NaN
336,29.0,0.0,66.6000,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN
605,36.0,0.0,15.5500,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN
193,3.0,1.0,26.0000,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
